In [1]:
import pandas as pd
import numpy as np
import random
import gc

In [2]:
train = pd.read_pickle('Data/train_data.pkl')

In [3]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120',
                'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
num_cols = set(train.columns).difference(set(cat_cols))

In [4]:
features = random.choices(list(num_cols), k=2)

In [5]:
train.drop(cat_cols, axis=1, inplace=True)

In [20]:
corr_df = train[features].groupby('customer_ID').corr()
corr_df

S_5       S_6
customer_ID                                                               
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb... S_5  1.000000 -0.112958
                                                   S_6 -0.112958  1.000000
00000fd6641609c6ece5454664794f0340ad84dddce9a26... S_5  1.000000 -0.158492
                                                   S_6 -0.158492  1.000000
00001b22f846c82c51f6e3958ccd81970162bae8b007e80... S_5  1.000000  0.323759
...                                                          ...       ...
ffff9984b999fccb2b6127635ed0736dda94e544e67e026... S_6 -0.164337  1.000000
ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf388145... S_5  1.000000 -0.254317
                                                   S_6 -0.254317  1.000000
fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eab... S_5  1.000000 -0.260805
                                                   S_6 -0.260805  1.000000

[917826 rows x 2 columns]

In [ ]:
index_list = []
tfc_list = []
for counter, id in enumerate(train.index.unique()):
    if counter%10000 == 0: print(counter, end=',')
    tfc = corr_df.loc[id].copy()
    tfc.fillna(0, inplace=True)
    tfc = tfc.mask(np.tril(np.ones(tfc.shape)).astype(bool))
    tfc_list.append(tfc.stack().values)
    index_list.append(id)

In [ ]:
study_df = pd.DataFrame(tfc_list, index=index_list)
study_df.to_pickle('Data/train_feature_corr.pkl')

In [ ]:
study_df

In [ ]:
scaler = StandardScaler()
study_df_scaled = scaler.fit_transform(study_df)
pca = PCA(n_components=50, random_state=42)
study_df_scaled_PCA = pca.fit_transform(study_df_scaled)
print(pca.explained_variance_ratio_)

In [ ]:
train_label = pd.read_pickle('Data/train_labels.pkl').loc[study_df.index]

In [ ]:
column_name = []
for i in range(0, study_df_scaled_PCA.shape[1], 1):
    column_name.append('PCA_FE_CORR_' + str(i))

pca_df = pd.DataFrame(study_df_scaled_PCA, columns=column_name, index=study_df.index)
feature_list = pca_df.columns
pca_target = pd.concat([pca_df, train_label], axis=1)

In [ ]:
correlation_dictionary_ft = associations(pca_target, nominal_columns = 'target',
                                        numerical_columns = feature_list.to_list(), mark_columns = True, 
                                        display_rows = ['target'], nan_strategy = 'drop_samples', 
                                        figsize= (15, 15), vmin = 0, vmax=0.8, compute_only = True, cbar = False)

In [ ]:
cft = correlation_dictionary_ft['corr'].stack().sort_values(ascending = False)
cft

In [ ]:
pca_df.to_pickle('Data/pca_fe_corr_train.pkl')